In [ ]:
system('wget https://github.com/jgm/pandoc/releases/download/2.13/pandoc-2.13-1-amd64.deb', intern=TRUE)
system('sudo dpkg -i ./pandoc-2.13-1-amd64.deb', intern = TRUE)

character(0)

[1] "(Reading database ... 160690 files and directories currently installed.)"
[2] "Preparing to unpack ./pandoc-2.13-1-amd64.deb ..."                       
[3] "Unpacking pandoc (2.13-1) over (1.19.2.4~dfsg-1build4) ..."              
[4] "Setting up pandoc (2.13-1) ..."                                          
[5] "Processing triggers for man-db (2.8.3-2ubuntu0.1) ..."

In [ ]:
install.packages('rmarkdown')
install.packages('tinytex')
install.packages('doParallel')
install.packages('ggplot2')
install.packages('torch')
install.packages('kableExtra')
install.packages('randomForest')
install.packages('randomForestSRC')
install.packages('ranger')
tinytex::install_tinytex()

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

tlmgr option sys_bin ~/bin



In [5]:
rmarkdown::render('Projet_JL_ZR_AM.Rmd', 'html_document', clean = FALSE)



processing file: Projet_JL_ZR_AM.Rmd



  |.                                                                     |   1%
  ordinary text without R code

  |..                                                                    |   3%
label: global_options (with options) 
List of 1
 $ include: logi FALSE

  |...                                                                   |   4%
label: setup (with options) 
List of 1
 $ include: logi FALSE

  |....                                                                  |   6%
  ordinary text without R code

  |.....                                                                 |   7%
label: unnamed-chunk-1
  |......                                                                |   8%
  ordinary text without R code

  |.......                                                               |  10%
label: unnamed-chunk-2 (with options) 
List of 4
 $ echo     : logi FALSE
 $ fig.cap  : chr "Moores Law Illustration"
 $ out.width: chr "80%"
 $ fig.align: chr "center"

  |........     

output file: Projet_JL_ZR_AM.knit.md




/usr/bin/pandoc +RTS -K512m -RTS Projet_JL_ZR_AM.utf8.md --to html4 --from markdown+autolink_bare_uris+tex_math_single_backslash --output Projet_JL_ZR_AM.html --lua-filter /usr/local/lib/R/site-library/rmarkdown/rmarkdown/lua/pagebreak.lua --lua-filter /usr/local/lib/R/site-library/rmarkdown/rmarkdown/lua/latex-div.lua --self-contained --variable bs3=TRUE --standalone --section-divs --template /usr/local/lib/R/site-library/rmarkdown/rmd/h/default.html --no-highlight --variable highlightjs=1 --variable theme=bootstrap --include-in-header /tmp/RtmpsDndps/rmarkdown-str492f4bffe9.html --mathjax --variable 'mathjax-url:https://mathjax.rstudio.com/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML' --citeproc 



Output created: Projet_JL_ZR_AM.html



In [ ]:
rmarkdown::find_pandoc(cache = FALSE)

$version
[1] ‘2.13’

$dir
[1] "/usr/bin"


In [ ]:
library(torch)
device_big_ops = "cpu"

In [ ]:
nb_heroes = 113
colnames_hero = paste0('H',1:nb_heroes)
colnames_df = c('Win', 'Cluster', 'GameMode', 'GameType',  colnames_hero)
data_train = read.csv('dota2Train.csv', col.names = colnames_df)

In [ ]:
data_heroes = data_train[,-(1:4)]
heroes_mat = as.matrix(data_heroes)
heroes_mat_all = rbind(heroes_mat > 0, heroes_mat < 0)
class(heroes_mat_all) = 'numeric'
heroes_mat_all[1:10,1:10]

H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,0,0,1,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0
0,1,0,0,1,1,0,0,0,0


In [ ]:
compare_heroes_torch = function(data_heroes, device_big_ops){
  # convert dataframe to tensor
  heroes_mat_all = torch_tensor(data_heroes, device = "cpu", dtype=torch_float32())
  tensor_heroes = heroes_mat_all$to(device=device_big_ops)
  
  #Execute the einsum
  a = torch_einsum('gi,gj,gk->ijk', list(tensor_heroes, tensor_heroes, tensor_heroes))
  
  #Find position of the argmax while reshaping the result tensor
  shape_tot = torch_flatten(a)$shape
  pos = torch_arange(1, shape_tot+1, device = device_big_ops)
  i = torch_floor_divide(pos, (nb_heroes*nb_heroes))
  j = torch_floor_divide(pos-i*nb_heroes*nb_heroes, nb_heroes)
  k = pos - i * nb_heroes * nb_heroes - j * nb_heroes
  pos = torch_vstack(c(pos, i+1 , j+1, k, torch_flatten(a)))
  
  # Take only the upper pyramidal part of the tensor to avoid repetitions
  pos = pos[,pos[2,]<pos[3,]]
  pos = pos[,pos[3,]<pos[4,]]
  
  pos_max = torch_argmax(pos[5,])
  max_val = pos[,pos_max]
  return(max_val)
}

#Time elapsed calculations
time_func = system.time(final_matrix_apply <- compare_heroes_torch(heroes_mat_all, "cpu"))
time_5 = time_func[3]
print(time_5)
print(final_matrix_apply)

elapsed 
  5.076 
torch_tensor
 103725
      9
     14
    104
    502
[ CPUFloatType{5} ]


In [ ]:
#Time elapsed calculations
time_func = system.time(final_matrix_apply <- compare_heroes_torch(heroes_mat_all, "cuda"))
time_5 = time_func[3]
print(time_5)
print(final_matrix_apply)

elapsed 
  0.373 
torch_tensor
 103725
      9
     14
    104
    502
[ CUDAFloatType{5} ]
